<a href="https://colab.research.google.com/github/psgareja/Complete-Python-3-Bootcamp/blob/master/BMMoviews_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

In [5]:
uploaded=files.upload()

In [0]:
uploaded=files.upload()

In [5]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 


    100% |████████████████████████████████| 592.3MB 94.4MB/s 


In [6]:
!pip3 install torchvision


    100% |████████████████████████████████| 61kB 3.7MB/s 
    100% |████████████████████████████████| 2.0MB 13.3MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
  [PIL]
You must restart the runtime in order to use newly installed versions.


In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [10]:
uploaded=files.upload()

Saving movies.dat to movies.dat


In [0]:
movies=pd.read_csv('movies.dat',sep='::',header=None,engine='python',encoding='latin-1')

In [12]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [13]:
uploaded=files.upload()

Saving users.dat to users.dat


In [0]:
users=pd.read_csv('users.dat',sep='::',header=None,engine='python',encoding='latin-1')

In [15]:
uploaded=files.upload()

Saving ratings.dat to ratings.dat


In [0]:
ratings=pd.read_csv('ratings.dat',sep='::',header=None,engine='python',encoding='latin-1')

In [22]:
uploaded=files.upload()

Saving u1.base to u1.base


In [0]:
training_set = pd.read_csv('u1.base', delimiter = '\t')


In [0]:
training_set=pd.read_csv('u1.base',delimiter='\t')

In [25]:
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [0]:
training_set=np.array(training_set,dtype='int')

In [28]:
uploaded=files.upload()

Saving u1.test to u1 (1).test


In [0]:
test_set=pd.read_csv('u1.test',delimiter='\t')
test_set=np.array(test_set,dtype='int')

In [0]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [31]:
nb_users

943

In [0]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)
    

In [0]:
training_set=torch.FloatTensor(training_set)
test_set=torch.FloatTensor(test_set)

In [34]:
training_set


    0     3     4  ...      0     0     0
    4     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
       ...          ⋱          ...       
    5     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
    0     5     0  ...      0     0     0
[torch.FloatTensor of size 943x1682]

In [35]:
test_set


    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
[torch.FloatTensor of size 943x1682]

In [0]:
training_set[training_set==0]=-1
training_set[training_set==1]=0
training_set[training_set==2]=0
training_set[training_set>=3]=1

In [0]:
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1


In [78]:
# Boltzmann Machines

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importing the dataset
movies = pd.read_csv('movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

# Preparing the training set and the test set
training_set = pd.read_csv('u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

# Creating the architecture of the Neural Network
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
    
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)
# Training the RBM
nb_epoch = 100
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

# Testing the RBM
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:297: UserWarning: other is not broadcastable to self, but they have the same number of elements.  Falling back to deprecated pointwise behavior.
  return self.add_(other)


epoch: 1 loss: 0.2921283905676625
epoch: 2 loss: 0.2521064492115036
epoch: 3 loss: 0.25302121261098454
epoch: 4 loss: 0.24947063927943397
epoch: 5 loss: 0.2501172203916919
epoch: 6 loss: 0.24989440448973818
epoch: 7 loss: 0.25197645820554826
epoch: 8 loss: 0.24724193534282377
epoch: 9 loss: 0.2494359842341578
epoch: 10 loss: 0.25159056229880433
epoch: 11 loss: 0.2500027599594465
epoch: 12 loss: 0.25013740359489395
epoch: 13 loss: 0.24998310095665
epoch: 14 loss: 0.25106083270563695
epoch: 15 loss: 0.25056815309857544
epoch: 16 loss: 0.2519077289636091
epoch: 17 loss: 0.24935690229465315
epoch: 18 loss: 0.25218714525470604
epoch: 19 loss: 0.24886425571381934
epoch: 20 loss: 0.2523145431539942
epoch: 21 loss: 0.2496278700285354
epoch: 22 loss: 0.24930608644324645
epoch: 23 loss: 0.24990576113452936
epoch: 24 loss: 0.25024068005579836
epoch: 25 loss: 0.25004936639130515
epoch: 26 loss: 0.24831479365023382
epoch: 27 loss: 0.25086701916671295
epoch: 28 loss: 0.250224880328992
epoch: 29 loss